I used all the presidential speeches by Clinton and Obama during their presidetial careers. Data adopted from [Grammar Lab](http://www.thegrammarlab.com/?nor-portfolio=corpus-of-presidential-speeches-cops-and-a-clintontrump-corpus) 

In [1]:
!pip install gensim

In [2]:
#library import code copied from https://github.com/tolga-b/debiaswe
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

In [3]:
#loading spacy and tokenizer
import spacy

import en_core_web_sm
nlp = en_core_web_sm.load()
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize

import datetime
import datefinder 
import string
import re
from nltk.tokenize import regexp_tokenize
from random import shuffle,uniform
from math import e , pow , log
import gensim
from gensim.models import Word2Vec

In [4]:
#using NLTK library for tokenizing all words from english text

def clean_text(text):
    
    #text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    speech_date=[]
    
    try:
        matches = datefinder.find_dates(text)
        for date in matches:
            speech_date.append(date)
    except:
        None
    finally:
        text = re.sub('[\d\n]', ' ', text)
        return speech_date[0],text

In [79]:
#importing speeches

# The absolute path of the current directory
path = 'speeches/'
import os

dir_base_clinton = "speeches/clinton"
dir_base_obama =  path + "/obama"

In [6]:
#function to read files

def read_file(filename):
    input_file_text = open(filename, encoding='latin1').read()
    return input_file_text

    
def read_directory_files(directory):
    file_texts = []
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    for f in files:
        file_text = read_file(os.path.join(directory, f))
        file_texts.append({"file":f, "content": file_text })
    return file_texts

In [7]:
#calling funtion to read the files
text_corpus_obama = read_directory_files(dir_base_obama)
text_corpus_clinton = read_directory_files(dir_base_clinton)

In [8]:
# #Converting list to string
# clinton_corpus = ' '.join([str(elem) for elem in text_corpus_clinton])
# obama_corpus = ' '.join([str(elem) for elem in text_corpus_obama])
# text_corpus_clinton

In [9]:
# #name, president served, start date, end date

# directors_dict = {'David Kusnet' : ['Bill Clinton','1993-01-20', '1994-12-31'],
# 'Michael Waldman' : ['Bill Clinton','1995-01-01', '1999-08-09'], 
# 'Terry Edmonds' : ['Bill Clinton', '1999-08-10', '2001-01-19'],
# 'Michael Gerson':['George W. Bush', '2001-01-20', '2005-02-12'],
# 'William McGurn':['George W. Bush', '2005-02-13', '2007-12-13'],
# 'Marc Thiessen': ['George W. Bush', '2007-12-14', '2009-01-20'], 
# 'Jon Favreau':['Barack Obama', '2009-01-21', '2013-03-01'], 
# 'Cody Keenan':['Barack Obama', '2013-03-02', '2017-01-20'], 
# 'Stephen Miller':['Donald Trump', '2017-01-21', '2021-01-20']}

In [91]:
#name, president served, start date, end date

directors_dict = {'David Kusnet' : ['Bill Clinton','1993-01-20', '1994-12-31'],
'Michael Waldman' : ['Bill Clinton','1995-01-01', '1999-08-09'], 
'Terry Edmonds' : ['Bill Clinton', '1999-08-10', '2001-01-19'], 
'Jon Favreau':['Barack Obama', '2009-01-21', '2013-03-01'], 
'Cody Keenan':['Barack Obama', '2013-03-02', '2017-01-20']}


Sources:

David Kusnet - https://prospect.org/topics/david-kusnet/ <br>
Michael Waldman - https://www.brennancenter.org/experts/michael-waldman <br>
Terry Edmonds - https://www.baltimoresun.com/news/bs-xpm-1999-08-10-9908100181-story.html <br>
Jon Favreau - https://www.google.com/search?q=jon+favreau+named+speechwriter&ei=iQhyYK2dI5ji5NoPtc6gmAE&oq=jon+favreau+named+speechwriter&gs_lcp=Cgdnd3Mtd2l6EAMyBQgAEM0CMgUIABDNAjIFCAAQzQI6BAghEApQ-FhY22ZghGloAHAAeACAAXqIAfAHkgEDOS4zmAEAoAEBqgEHZ3dzLXdpesABAQ&sclient=gws-wiz&ved=0ahUKEwjthZWWwPTvAhUYMVkFHTUnCBMQ4dUDCA0&uact=5 <br>
Cody Keenan - https://www.theatlantic.com/politics/archive/2013/02/who-is-cody-keenan-obamas-sotu-speechwriter/272938/ <br>

**Now we have information of speech writers lets create 9 list of dictionaries that will have presidential speeches for each speech writers for Bill Clinton and Obama**

In [11]:
#trying to separate each speech to respective speech writer by each speech writer term
from datetime import datetime

def collect_speech_writers(doc, first_writer_start, first_writer_end, second_writer_start, second_writer_end, third_writer_start, third_writer_end, val):
    speech_date,cleaned_text = clean_text(doc)
    speech_date_str = speech_date.strftime("%Y-%m-%d")
    first_new_dict={}
    second_new_dict={}
    third_new_dict ={}
    try:
        if speech_date > first_writer_start and speech_date < first_writer_end:
            first_new_dict[speech_date_str]= cleaned_text
        elif speech_date > second_writer_start and speech_date < second_writer_end:
            second_new_dict[speech_date_str]= cleaned_text
        elif speech_date > third_writer_start and speech_date < third_writer_end:
            third_new_dict[speech_date_str]= cleaned_text
    except:
        None
    finally:
        return first_new_dict, second_new_dict,third_new_dict

In [12]:
# looping over clinton's text corpus to get collection of each speechwriter
import datetime

clinton_first_writer=[]
clinton_second_writer=[]
clinton_third_writer=[]

for i in range(len(text_corpus_clinton)):
    first_writer_start = datetime.datetime(1993, 1,20)
    first_writer_end = datetime.datetime(1995, 1, 12)
    second_writer_start = datetime.datetime(1995, 1, 12)
    second_writer_end = datetime.datetime(1999, 1, 12)
    third_writer_start = datetime.datetime(1999, 1, 12)
    third_writer_end = datetime.datetime(2001, 1, 20)
    clinton_first, clinton_sd, clinton_td = collect_speech_writers(text_corpus_clinton[i]['content'], first_writer_start, first_writer_end, second_writer_start, 
                                                 second_writer_end, third_writer_start, third_writer_end, i+1 )
    if len(clinton_first) !=0:
        clinton_first_writer.append(clinton_first)
    elif len(clinton_sd) !=0:
        clinton_second_writer.append(clinton_sd)
    elif len(clinton_td) !=0:
        clinton_third_writer.append(clinton_td)

In [13]:
# looping over obama's text corpus to get collection of each speechwriter
obama_first_writer=[]
obama_second_writer=[]

for i in range(len(text_corpus_obama)):
    first_writer_start = datetime.datetime(2009, 1,20)
    first_writer_end = datetime.datetime(2013, 3, 1)
    second_writer_start = datetime.datetime(2013, 3, 1)
    second_writer_end = datetime.datetime(2017, 2, 20)
    third_writer_start = datetime.datetime(2000, 1, 1)  #obama doesnot have third writer--this date is just makeup date to make the function work
    third_writer_end = datetime.datetime(2000, 1, 1)  #obama doesnot have third writer--this date is just makeup date to make the function work
    obama_first, obama_sd, obama_td = collect_speech_writers (text_corpus_obama[i]['content'], first_writer_start, first_writer_end, second_writer_start, second_writer_end, third_writer_start, third_writer_end, i+1 )
    if len(obama_first) !=0:
        obama_first_writer.append(obama_first)
    elif len(obama_sd) !=0:
        obama_second_writer.append(obama_sd)

In [14]:
def read_file(filename):
    input_file_text = open(filename, encoding='latin1').read()
    return input_file_text

    
def read_directory_files(directory):
    file_texts = []
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    
    for f in files:
        try:
            file_text = read_file(os.path.join(directory, f))
            
            if "Clinton" in f:
                file_texts.append({"file":f, "content": file_text })
        except:
            pass
    return file_texts

In [15]:
def collect_speech_writers(doc, first_start, first_end, second_start, second_end, third_start, third_end, val):
    speech_date, cleaned_text = clean_text(doc)
    speech_date_str = speech_date.strftime("%Y-%m-%d")
    
    first_dict={}
    second_dict={}
    third_dict={}
    
    try:
        if speech_date > first_start and speech_date < first_end: first_dict[speech_date_str]= cleaned_text
        elif speech_date > second_start and speech_date < second_end: second_dict[speech_date_str]= cleaned_text
        elif speech_date > third_start and speech_date < third_end: third_dict[speech_date_str]= cleaned_text
    
    except:
        None
        
    finally:
        return first_dict, second_dict, third_dict

In [16]:
clinton_dir = {'David Kusnet' : ['Bill Clinton','1993-01-20', '1994-12-31'],
               'Michael Waldman' : ['Bill Clinton','1995-01-01', '1999-08-09'],
               'Terry Edmonds' : ['Bill Clinton', '1999-08-10', '2001-01-19']}

In [17]:
def collect_speech_of_all_writer(x): #pass the list of all dictionaries of speeches written by each writer
    speech = ' '
    for i in range(len(x)):
        speech = speech +' ' + "".join(x[i].values())
    return speech

In [18]:
#get list of all dictionarites of each speech writer
all_speech_of_Clinton_first_writer = collect_speech_of_all_writer(clinton_first_writer)
all_speech_of_Clinton_second_writer = collect_speech_of_all_writer(clinton_second_writer)
all_speech_of_Clinton_third_writer = collect_speech_of_all_writer(clinton_third_writer)

all_speech_of_Obama_first_writer = collect_speech_of_all_writer(obama_first_writer)
all_speech_of_Obama_second_writer = collect_speech_of_all_writer(obama_second_writer)

In [19]:
all_speech_of_Clinton_first_writer

'  <title="Press Conference on Gays in the Military\'">  <date="January   ,     ">  <The President.> Good afternoon, ladies and gentlemen. I\'m sorry, we had a last-minute delay occasioned by another issue, not this one. The debate over whether to lift the ban on homosexuals in the military has, to put it mildly, sparked a great deal of interest over the last few days. Today, as you know, I have reached an agreement, at least with Senator Nunn and Senator Mitchell, about how we will proceed in the next few days. But first I would like to explain what I believe about this issue and why, and what I have decided to do after a long conversation, and a very good one, with the Joint Chiefs of Staff and discussions with several Members of Congress. The issue is not whether there should be homosexuals in the military. Everyone concedes that there are. The issue is whether men and women who can and have served with real distinction should be excluded from military service solely on the basis of

In [54]:
#converting to list
speechs_of_Clinton_first_writer = [all_speech_of_Clinton_first_writer]
speechs_of_Clinton_second_writer = [all_speech_of_Clinton_second_writer]
speechs_of_Clinton_third_writer = [all_speech_of_Clinton_third_writer]
speechs_of_Obama_first_writer = [all_speech_of_Obama_first_writer]
speechs_of_Obama_second_writer = [all_speech_of_Obama_second_writer]

In [92]:
#converting speeches to word2vec 
clinton_1st_word2vec = gensim.models.Word2Vec(speechs_of_Clinton_first_writer, min_count = 1, vector_size = 100, window = 5)
# obama_word_2vec = gensim.models.Word2Vec(text_corpus_obama, min_count = 1, vector_size = 100, window = 5)

In [62]:
clinton_1st_word2vec

In [63]:
clinton_1st_word2vec.wv.save_word2vec_format('clinton1st_writer.txt', binary = False)
clinton_1st_word2vec.save("word2vec_clinton.model")
# obama_word_2vec.wv.save_word2vec_format('obama.txt', binary = False)

**Loading data**

In [66]:
# load google news word2vec
#code in this cell adopted from https://github.com/tolga-b/debiaswe
E = WordEmbedding('clinton1st_writer.txt')

*** Reading data from clinton1st_writer.txt
Got weird line e -0.109417215 -0.14523052 0.19111119 -0.02213058 -0.19801289 -0.22061855 0.025427677 0.35960737 -0.23809718 -0.21323372 0.07991371 -0.1821514 -0.19697568 0.17739283 -0.08222524 -0.07145223 0.031031815 0.060850065 -0.21609993 -0.32133165 0.17488214 0.109305404 0.43776166 0.067016795 0.12790278 0.03417102 0.1516855 0.20265572 -0.25648618 0.014867178 -0.024194838 -0.09120099 0.10160918 -0.40193155 0.015977088 -0.0070958734 0.33950344 -0.18176186 -0.18356058 -0.0839553 -0.18705238 0.04162816 -0.29598385 -0.1087234 0.16174857 0.0010214925 -0.2737226 0.25009972 0.10401356 0.17377876 -0.15268406 0.006752441 -0.052608185 0.0065193195 0.01401405 -0.082602166 0.03064678 -0.21943237 -0.13366875 0.19151162 0.06789362 0.032175556 0.057357717 -0.06435559 -0.043012064 0.12677817 -0.069388986 0.32282656 -0.05708496 0.09756733 0.311194 0.38676652 0.053181633 -0.04898121 0.1150731 0.047840334 0.26901904 -0.13116318 -0.050410148 -0.17411804 -0.0

In [28]:
# D = WordEmbedding('obama.txt')

In [26]:
# size = 300 #word vector size
# word = 'food' #word token

# try:
#     wordVector = model[word].reshape((1, size))
# except KeyError:
#     print "not found! ",  word

In [29]:
#define gender direction
# v_gender_obama = D.diff('she','he')

In [67]:
#Define Gender Direction

v_gender_clinton = E.diff('he','she')

KeyError: 'he'

In [ ]:
# analogies gender
a_gender_2 = E.best_analogies_dist_thresh(v_gender_2)

for (a,b,c) in a_gender_2:
    print(a+"-"+b)

In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
# # iterate through each sentence for obama in the file

# clinton = []

# for i in sent_tokenize(clinton_s):
#     temp = []

#     # tokenize the sentence into words
#     for j in word_tokenize(i):
#         temp.append(j.lower())

#     clinton.append(temp)

In [ ]:
# # iterate through each sentence for obama in the file

# obama = []

# for i in sent_tokenize(clinton_s):
#     temp = []

#     # tokenize the sentence into words
#     for j in word_tokenize(i):
#         temp.append(j.lower())

#     obama.append(temp)

In [ ]:
# v_gender = model1.diff('she','he')